In [79]:
import os
os.system('make exportdb')

0

In [80]:
import pandas as pd
links = pd.read_csv('data/export.csv', header=None)
ALL_COLS = ['id', 'url', 'title', 'summary', 'domain', 'added', 'modified',
            'liked', 'category', 'aggregator', 'seed']
links.columns = ALL_COLS
links['domain'] = links['domain'].apply(lambda s: str(s).replace('.', '').replace('www', ''))
links['url2'] = links['url'].apply(lambda s: str(s).replace('.', ' ').replace('/', ' ').replace(':', '').replace('_', ' ').replace('-', ' '))
links['text'] = links['url2'] + ' ' + links['title']
links = links.drop(['summary', 'added', 'modified', 'seed', 'category'], axis=1)
links[['aggregator', 'text', 'liked']]

,aggregator,text,liked
0,custom,http moralitylab bc edu wp content uploads 20...,1.0
1,custom,http www jefftk com news 2013 04 01 The Unint...,1.0
2,custom,http calnewport com blog 2013 03 24 how to wr...,1.0
3,custom,http www freakonomics com 2012 10 31 an alter...,1.0
4,custom,http www wired com vanish 2009 11 ff vanish2 ...,1.0
...,...,...,...
58317,Vox,https www vox com 2020 6 23 21296086 new york...,NaN
58318,Schneier,https www schneier com blog archives 2020 06 ...,NaN
58319,Vox,https www vox com world 21273709 brazil coron...,0.0
58320,538,https fivethirtyeight com videos do you buy t...,1.0


In [81]:
links['liked'].value_counts()

 0.0    3170
 1.0    2918
-1.0      13
Name: liked, dtype: int64

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1, 1),
                        max_features=5000,
                        min_df=2,
                        max_df=0.8,
                        binary=True)
links_ = links[(links['liked'] == 0) | (links['liked'] == 1)]
tfidf_train = tfidf.fit_transform(links_['text'])
tfidf_train = pd.DataFrame(tfidf_train.toarray(), columns=['vocab_{}'.format(v) for v in tfidf.get_feature_names()])
tfidf_train

,vocab_000,vocab_00pm,vocab_01,vocab_02,vocab_020,vocab_03,vocab_039,vocab_04,vocab_05,vocab_06,...,vocab_zealand,vocab_zen,vocab_zenhabits,vocab_zero,vocab_zfs,vocab_zhrpjdc3cepseebue,vocab_zone,vocab_zoom,vocab_zsh,vocab_zuckerberg
0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.207607,0.0,0.0,0.000000,0.0,0.260419,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.182698,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6083,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6084,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6085,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6086,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore')
ag_train = ohe.fit_transform(links_[['aggregator']])
ag_train = pd.DataFrame(ag_train.toarray(), columns=['ag_{}'.format(a) for a in ohe.get_feature_names()])
ag_train

,ag_x0_127,ag_x0_3P,ag_x0_538,ag_x0_80K,ag_x0_AI Impacts,ag_x0_ALOP,ag_x0_Aarora,ag_x0_Acton,ag_x0_AllegedWisdom,ag_x0_Archive,...,ag_x0_VeganStrat,ag_x0_Vox,ag_x0_VoxEU,ag_x0_WBW,ag_x0_WTB,ag_x0_Whittlestone,ag_x0_WorldInData,ag_x0_YaschaMounk,ag_x0_culture,ag_x0_custom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6085,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6086,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
train = pd.concat((tfidf_train.reset_index(drop=True), ag_train.reset_index(drop=True)), axis=1)
train

,vocab_000,vocab_00pm,vocab_01,vocab_02,vocab_020,vocab_03,vocab_039,vocab_04,vocab_05,vocab_06,...,ag_x0_VeganStrat,ag_x0_Vox,ag_x0_VoxEU,ag_x0_WBW,ag_x0_WTB,ag_x0_Whittlestone,ag_x0_WorldInData,ag_x0_YaschaMounk,ag_x0_culture,ag_x0_custom
0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.207607,0.0,0.0,0.000000,0.0,0.260419,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.000000,0.0,0.0,0.182698,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6083,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6084,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6085,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6086,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
from mlgear.cv import run_cv_model
from mlgear.utils import print_step

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score as auc
from sklearn.preprocessing import StandardScaler

def runLR(train_X, train_y, test_X=None, test_y=None, test_X2=None, params={}, meta=None):
    if not params.get('random_state'):
        params['random_state'] = 42

    print_step('Train LR')
    model = LogisticRegression(**params)
    model.fit(train_X, train_y)
    if test_X is not None:
        print_step('Predict 1/2')
        pred_test_y = model.predict_proba(test_X)[:, 1]
    else:
        pred_test_y = None
    if test_X2 is not None:
        print_step('Predict 2/2')
        pred_test_y2 = model.predict_proba(test_X2)[:, 1]
    else:
        pred_test_y2 = None
    return pred_test_y, pred_test_y2, model.coef_[0], model


results = run_cv_model(train.values, None, links_['liked'].values, runLR, {}, auc, train_on_full=True)

Started model fold 1/5
[2020-06-23 11:36:53.614361] Train LR
[2020-06-23 11:36:56.041559] Predict 1/2
model cv score 1: 0.8981968241606092
Started model fold 2/5
[2020-06-23 11:36:56.695877] Train LR
[2020-06-23 11:36:58.552735] Predict 1/2
model cv score 2: 0.9026368013533737
Started model fold 3/5
[2020-06-23 11:36:59.175071] Train LR
[2020-06-23 11:37:01.447780] Predict 1/2
model cv score 3: 0.8871104237583712
Started model fold 4/5
[2020-06-23 11:37:02.105370] Train LR
[2020-06-23 11:37:03.809489] Predict 1/2
model cv score 4: 0.9085265034851678
Started model fold 5/5
[2020-06-23 11:37:04.422716] Train LR
[2020-06-23 11:37:06.135874] Predict 1/2
model cv score 5: 0.9073482384970236
## Training on full ##
[2020-06-23 11:37:06.151076] Train LR
model cv scores : [0.8981968241606092, 0.9026368013533737, 0.8871104237583712, 0.9085265034851678, 0.9073482384970236]
model cv mean score : 0.900763758250909
model cv total score : 0.9002699982486602
model cv std score : 0.007747600629049448


In [91]:
prod_model = results['model']['full']
importances = pd.DataFrame({'variables': train.columns, 'importance': prod_model.coef_[0]})
show(importances.sort_values('importance'), max_rows=None)

                              variables  importance
5126                          ag_x0_TDS   -2.187681
5049              ag_x0_Freakonometrics   -2.134654
9                              vocab_06   -2.094400
2056                       vocab_github   -2.028947
5062                           ag_x0_HN   -1.971184
5132                        ag_x0_VoxEU   -1.958984
5033                       ag_x0_DeLong   -1.731716
942                      vocab_comments   -1.662115
4778                          vocab_vox   -1.507131
5088                           ag_x0_NR   -1.453871
1825                    vocab_feedproxy   -1.426053
5055                    ag_x0_Greenspun   -1.387977
4342                       vocab_system   -1.366008
5089              ag_x0_NakedCapitalism   -1.355870
5068                        ag_x0_Kling   -1.338499
1127                      vocab_culture   -1.309046
3040                         vocab_news   -1.308553
5061                          ag_x0_HBR   -1.257541
3331        

In [87]:
links_pred = links[(links['liked'] != 0) & (links['liked'] != 1) & (links['liked'] != -1)]
links_pred = links_pred.drop(['liked', 'domain'], axis=1)
links_pred

,id,url,title,aggregator,url2,text
18,14992,http://sebastianmarshall.com/80-or-never,80% or never,Archives,http sebastianmarshall com 80 or never,http sebastianmarshall com 80 or never 80% or...
62,5207,https://dariusforoux.com/one-thing/,https://dariusforoux.com/one-thing/,Archives,https dariusforoux com one thing,https dariusforoux com one thing https://dar...
116,31354,https://fakenous.net/?p=1302,We Are Doomed,FakeNous,https fakenous net ?p=1302,https fakenous net ?p=1302 We Are Doomed
260,18880,https://zenhabits.net/gollum,How to Love Your Dark Side,Leo,https zenhabits net gollum,https zenhabits net gollum How to Love Your D...
279,6994,http://danluu.com/randomize-hn/,Why HN should use randomized algorithms,Archives,http danluu com randomize hn,http danluu com randomize hn Why HN should u...
...,...,...,...,...,...,...
58312,95094,https://reason.com/2020/06/23/seattle-says-it-...,Seattle Says It Will Ask Protesters To Volunta...,Reason,https reason com 2020 06 23 seattle says it w...,https reason com 2020 06 23 seattle says it w...
58313,95096,https://blog.adamretter.org.uk/archival-identi...,Archival Identifiers for Digital Files,HN,https blog adamretter org uk archival identif...,https blog adamretter org uk archival identif...
58314,95097,https://fivethirtyeight.com/features/todays-el...,Today’s Elections In Kentucky And New York Are...,538,https fivethirtyeight com features todays ele...,https fivethirtyeight com features todays ele...
58317,95098,https://www.vox.com/2020/6/23/21296086/new-yor...,15 primary races to watch in New York,Vox,https www vox com 2020 6 23 21296086 new york...,https www vox com 2020 6 23 21296086 new york...


In [88]:
tfidf_test = tfidf.transform(links_pred['text'])
tfidf_test = pd.DataFrame(tfidf_test.toarray(), columns=['vocab_{}'.format(v) for v in tfidf.get_feature_names()])
tfidf_test

,vocab_000,vocab_00pm,vocab_01,vocab_02,vocab_020,vocab_03,vocab_039,vocab_04,vocab_05,vocab_06,...,vocab_zealand,vocab_zen,vocab_zenhabits,vocab_zero,vocab_zfs,vocab_zhrpjdc3cepseebue,vocab_zone,vocab_zoom,vocab_zsh,vocab_zuckerberg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.386745,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.138580,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.289951,0.0,0.0,0.0
52217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
52218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
52219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [89]:
ag_test = ohe.transform(links_pred[['aggregator']])
ag_test = pd.DataFrame(ag_test.toarray(), columns=['ag_{}'.format(a) for a in ohe.get_feature_names()])
ag_test

,ag_x0_127,ag_x0_3P,ag_x0_538,ag_x0_80K,ag_x0_AI Impacts,ag_x0_ALOP,ag_x0_Aarora,ag_x0_Acton,ag_x0_AllegedWisdom,ag_x0_Archive,...,ag_x0_VeganStrat,ag_x0_Vox,ag_x0_VoxEU,ag_x0_WBW,ag_x0_WTB,ag_x0_Whittlestone,ag_x0_WorldInData,ag_x0_YaschaMounk,ag_x0_culture,ag_x0_custom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52218,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
test = pd.concat((tfidf_test.reset_index(drop=True), ag_test.reset_index(drop=True)), axis=1)
test

,vocab_000,vocab_00pm,vocab_01,vocab_02,vocab_020,vocab_03,vocab_039,vocab_04,vocab_05,vocab_06,...,ag_x0_VeganStrat,ag_x0_Vox,ag_x0_VoxEU,ag_x0_WBW,ag_x0_WTB,ag_x0_Whittlestone,ag_x0_WorldInData,ag_x0_YaschaMounk,ag_x0_culture,ag_x0_custom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.138580,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
preds = prod_model.predict_proba(test)[:, 1]
preds

array([0.23957008, 0.23820345, 0.54851777, ..., 0.59650134, 0.24376944,
       0.26410586])

In [105]:
links_pred['pred'] = preds
links_display = links_pred[['id', 'url', 'title', 'aggregator', 'pred']].sort_values('pred', ascending=False)
links_display.head(50)

,id,url,title,aggregator,pred
34058,86510,https://softwareleadweekly.com/issues/100,Software Lead Weekly Issue #100,SLW,0.946486
17970,21228,https://www.washingtonpost.com/politics/2020/0...,How the U.S.-Iran conflict could help China an...,Twitter,0.945176
20794,25761,https://policytensor.com/2020/01/20/how-did-tr...,How Did Trump Win?,Twitter,0.939596
40255,25589,https://blogs.sciencemag.org/pipeline/archives...,Coronavirus,Twitter,0.936705
33473,86351,https://softwareleadweekly.com/issues/160,Software Lead Weekly Issue #160,SLW,0.935385
21287,26701,https://blog.givewell.org/2012/08/23/how-we-ev...,How We Evaluate a Study,Custom,0.934692
26250,32534,https://slatestarcodex.com/2020/02/19/links-2-20/,Links 2/20,SSC,0.934244
19285,23485,https://reason.com/2020/01/20/google-v-oracle/,Google v. Oracle,Twitter,0.932164
33905,86464,https://softwareleadweekly.com/issues/101,Software Lead Weekly Issue #101,SLW,0.931602
14205,16496,https://www.nytimes.com/2020/01/03/opinion/ira...,Trump Kills Iran’s Most Overrated Warrior,Twitter,0.930283


In [106]:
links_display.tail(50)

,id,url,title,aggregator,pred
29015,35530,https://towardsdatascience.com/building-an-inc...,Building an Incremental Recommender System: Pa...,TDS,0.011400
55479,82712,https://github.com/ayoy/fontedit,Show HN: FontEdit – A desktop app to develop f...,HN,0.011317
25487,31574,https://github.com/laixintao/iredis/,Show HN: IRedis: CLI for Redis with AutoComple...,HN,0.011313
36364,43690,https://github.com/System-Glitch/goyave,Show HN: Goyave – Feature-complete and opinion...,HN,0.011285
24205,30057,https://github.com/mingrammer/diagrams,Diagram as Code for prototyping cloud system a...,HN,0.011227
27628,34124,https://towardsdatascience.com/recommendation-...,Recommendation System in Python: LightFM,TDS,0.011175
46705,54314,https://github.com/knguyenanhoa/cli-arxiv,Show HN: An interactive CLI ArXiv explorer,HN,0.011034
36204,43503,https://github.com/pm4py/pm4py-source,PM4Py – An Open Source Python Library for Proc...,HN,0.010872
42301,49676,https://github.com/yusufnb/verify-json,Show HN: Verify JSON using minimal schema,HN,0.010818
36982,44374,https://github.com/kgabis/ape,Show HN: Ape – an easy-to-embed programming la...,HN,0.010679
